In [1]:
from datetime import datetime

In [9]:
# from predict.py
def get_days_minutes(date_time):
    '''time_str'''
    return date_time.hour * 60 + date_time.minute

# Data Preprocessing for a single ride given as a dict?
def prepare_features(ride):
    ### add the current time in minutes to the ride data
    now_time = datetime.now()
    features = {}

    ### Transform given locatio_ids to a single trip route
    # syntax object.value following pydantic syntax
    features['trip_route'] = str(ride.PULocationID) + "_" + str(ride.DOLocationID)
    features["trip_distance"] = ride.trip_distance
    features["pickup_time_minutes"] = get_days_minutes(now_time)
    features["passenger_count"] = ride.passenger_count

    return features

# Load the Model from ML-Flow
def load_model(run_id):
    #stage = "Production"
    model_uri = f"runs:/{run_id}/model"
    model = mlflow.pyfunc.load_model(model_uri)
    return model

# Return a prediction
# To understand the handling of the input data f
def predict(model_name, data):
  
    load_dotenv("../.env")
    MLFLOW_TRACKING_URI = os.getenv("MLFLOW_TRACKING_URI")
    SA_KEY = os.getenv("SA_KEY")
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = SA_KEY
    
    mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
    model_input = prepare_features(data)
    model = load_model(model_name)
    prediction = model_input
    # prediction = model.predict(model_input) # preprocessing pipeline (vectorization and encoding) is part of the model
    return float(prediction["trip_distance"])   

In [10]:
from webservice_locally.data_model import TaxiRide, TaxiRidePrediction # data models imported
from webservice_locally.predict import predict

# from app.py
def predict_duration(data: TaxiRide): # Data beeing send in the model of TaxiRide 
    # (RideID, PU, DO, distance)
    prediction = predict("350d0a6fd4884d47983e1b2b244679f9", data) # makes pre-processing and prediction of
    # one instance 
    # In the Return-statement, the input-data of the post-method in the data-model of
    # TaxiRide is unpacked into the data-model of TaxiRidePrediction, which inherits from TaxiRide
    # so that the returned object holds the data about the ride and the predicted duration
    return TaxiRidePrediction(**data.dict(), 
                              predicted_duration=prediction)

In [11]:
predict_duration({
  "ride_id": "bla",
  "PULocationID": 1,
  "DOLocationID": 10,
  "trip_distance": 12,
  "passenger_count": 2
})

AttributeError: 'dict' object has no attribute 'PULocationID'